In [2]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D

In [3]:
import json
with open("./metadata/chunklist.json") as f:
    chunks = json.load(f) 
chunks.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'ZH', 'signal'])

In [4]:
chunks["signal"][0]

['/hadoop/cms/store/user/legianni/ProjectMetis/HHggtautau_Era2017____x3/test_nanoaodSkim_1.root',
 0,
 33831]

In [8]:
file = uproot.open(chunks["signal"][0][0])
tree = file['Events']
tree.keys()
#tree.show()
gen_keys = tree.keys(filter_name="GenPart_*")
genJet_keys = tree.keys(filter_name="GenJet_*")
gen_keys, genJet_keys

(['GenPart_eta',
  'GenPart_mass',
  'GenPart_phi',
  'GenPart_pt',
  'GenPart_genPartIdxMother',
  'GenPart_pdgId',
  'GenPart_status',
  'GenPart_statusFlags'],
 ['GenJet_eta',
  'GenJet_mass',
  'GenJet_phi',
  'GenJet_pt',
  'GenJet_partonFlavour',
  'GenJet_hadronFlavour'])

In [39]:
genPart = tree.arrays(gen_keys + genJet_keys + ["genWeight"], library="ak", how="zip") 
mask_genTau = abs(genPart.GenPart.pdgId) == 15
mask_genPho = abs(genPart.GenPart.pdgId) == 22 
mIdx = genPart.GenPart.genPartIdxMother
mask_fromH = genPart.GenPart.pdgId[mIdx] == 25

In [40]:
genTau_fromH = genPart.GenPart[mask_genTau & mask_fromH]
genPho_fromH = genPart.GenPart[mask_genPho & mask_fromH]

In [25]:
genTau_fromH.genPartIdxMother[:10]

<Array [[17, 17], [12, 12, ... 18], [12, 12]] type='10 * var * int32'>

In [41]:
Hist1D(ak.num(genPho_fromH), bins=np.linspace(0,5,6))

bin,content
"(0,1)",0 ± 0
"(1,2)",0 ± 0
"(2,3)",33831 ± 183.932
"(3,4)",0 ± 0
"(4,5)",0 ± 0


In [29]:
Hist1D(ak.num(genTau_fromH), bins=np.linspace(0,5,6))

bin,content
"(0,1)",0 ± 0
"(1,2)",0 ± 0
"(2,3)",33831 ± 183.932
"(3,4)",0 ± 0
"(4,5)",0 ± 0


In [33]:
import numba as nb
import math

@nb.jit
def deltaphi_devfunc(phi1, phi2):
    dphi = phi1 - phi2
    out_dphi = float(0)
    
    if dphi > math.pi:
        dphi = dphi - 2 * math.pi
        out_dphi = dphi
    elif (dphi + math.pi) < 0:
        dphi = dphi + 2 * math.pi
        out_dphi = dphi
    else:
        out_dphi = dphi
    return out_dphi

In [42]:
%%time

@nb.jit
def get_deltaR(taus):
    nEvents = len(taus)
    dRs = np.empty(nEvents, np.float32)
    for i in range(nEvents):
        tau1 = taus[i][0]
        tau2 = taus[i][1]
        dR = np.sqrt((tau1.eta-tau2.eta)**2 + deltaphi_devfunc(tau1.phi, tau2.phi)**2)  
        dRs[i] = dR
    return dRs

dRs_ditau = get_deltaR(genTau_fromH)
dRs_dipho = get_deltaR(genPho_fromH)

CPU times: user 227 ms, sys: 21.1 ms, total: 248 ms
Wall time: 246 ms


In [44]:
(1595+800)/33831

0.07079305961987527

In [43]:
Hist1D(dRs_dipho, bins=np.linspace(0,5,21))

[75.00 ± 8.66   807.00 ± 28.41   1591.00 ± 39.89   2471.00 ± 49.71
 3545.00 ± 59.54   4121.00 ± 64.20   4203.00 ± 64.83   3681.00 ± 60.67
 3262.00 ± 57.11   2759.00 ± 52.53   2314.00 ± 48.10   2158.00 ± 46.45
 1661.00 ± 40.76   604.00 ± 24.58   283.00 ± 16.82   142.00 ± 11.92
 66.00 ± 8.12   38.00 ± 6.16   19.00 ± 4.36   31.00 ± 5.57]

In [37]:
Hist1D(dRs_ditau, bins=np.linspace(0,5,21))

[76.00 ± 8.72   774.00 ± 27.82   1596.00 ± 39.95   2478.00 ± 49.78
 3509.00 ± 59.24   4057.00 ± 63.69   4109.00 ± 64.10   3580.00 ± 59.83
 3190.00 ± 56.48   2716.00 ± 52.12   2346.00 ± 48.44   2199.00 ± 46.89
 1791.00 ± 42.32   678.00 ± 26.04   337.00 ± 18.36   175.00 ± 13.23
 94.00 ± 9.70   56.00 ± 7.48   31.00 ± 5.57   39.00 ± 6.24]